In [14]:
import sys

import pandas as pd

sys.path.append("../")
from utils import *
from agoda_cancellation_estimator import *
import agoda_cancellation_preprocessor
from agoda_cancellation_prediction import *
from agoda_cancellation_preprocessor import AgodaCancellationPreprocessor
from sklearn.metrics import roc_curve, auc, f1_score, recall_score, \
    precision_score, classification_report, confusion_matrix
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier

import importlib

importlib.reload(agoda_cancellation_preprocessor)
from agoda_cancellation_preprocessor import *

np.random.seed(0)
c = [custom[0], custom[-1]]

# Load and preprocess data
full_data = load_data(
    "../datasets/agoda_cancellation_train.csv")
p = AgodaCancellationPreprocessor(full_data)
base_design_matrix, week_specific = p.preprocess(full_data)
cancellation_labels_list = p.preprocess_labels(full_data.cancellation_datetime,
                                          full_data.booking_datetime)
design_matrix = pd.DataFrame()
cancellation_labels = pd.DataFrame()
for i in range(len(week_specific)):
	pd.concat([design_matrix, pd.concat([base_design_matrix,week_specific[i]], axis=1)])
	pd.concat([cancellation_labels, cancellation_labels_list[i]])
for i in range(1, WEEK - 3):
	week_data = pd.read_csv(f"test_set_week_{i}.csv")
	test_set_i = p.preprocess(week_data)[0]
	test_set_i_labels = pd.read_csv(f"test_set_week_{i}_labels.csv")[
        "h_booking_id|label"].astype(str).apply(lambda x: int(x[-1]))
	p.add_week_features(test_set_i, week_data)
	design_matrix = pd.concat(
        [design_matrix, test_set_i])
	cancellation_labels = pd.concat(
        [cancellation_labels, test_set_i_labels])
	test_set_i = fill_missing_columns(design_matrix, test_set_i)

week_data = pd.read_csv(f"test_set_week_{WEEK-2}.csv")
train_test_set = p.preprocess(week_data)[0]
train_test_set_labels = pd.read_csv(f"test_set_week_{WEEK-2}_labels.csv")[
	"h_booking_id|label"].astype(str).apply(lambda x: int(x[-1]))
p.add_week_features(train_test_set, week_data)
train_test_set = fill_missing_columns(design_matrix, train_test_set)


week_data = pd.read_csv(f"test_set_week_{WEEK-1}.csv")
test_set = p.preprocess(week_data)[0]
test_set_labels = pd.read_csv(f"test_set_week_{WEEK-2}_labels.csv")[
	"h_booking_id|label"].astype(str).apply(lambda x: int(x[-1]))
p.add_week_features(test_set, week_data)
test_set = fill_missing_columns(design_matrix, test_set)


In [15]:
import agoda_cancellation_estimator
import importlib

importlib.reload(agoda_cancellation_estimator)
from agoda_cancellation_estimator import *

train_X = design_matrix
train_y = np.array(cancellation_labels).reshape((-1,))
test_X = test_set
test_y = np.array(test_set_labels).reshape((-1,))
train_test_X = train_test_set
train_test_y = train_test_set_labels

# Fit model over data with different model seeds.
scores = []
for n in range(5):
	np.random.seed(0)
	# model = AgodaEnsembleCancellationEstimator(train_test_set, train_test_set_labels, verbose=True).fit(
	# 	train_X, train_y)
	model=GradientBoostingClassifier().fit(train_X,train_y)
	y_pred = model.predict(test_X)
	f1 = f1_score(test_y, y_pred, labels=[0, 1], average="macro")
	precision = precision_score(test_y, y_pred, labels=[0, 1],
								average="macro")
	recall = recall_score(test_y, y_pred, labels=[0, 1], average="macro")
	print(
		f"f1 score: {f1}, precision score: {precision}, recall score: {recall}.")
	print(confusion_matrix(test_y, y_pred, labels=[0, 1]))
	y_pred = model.predict(train_test_X)
	f1 = f1_score(train_test_y, y_pred, labels=[0, 1], average="macro")
	precision = precision_score(train_test_y, y_pred, labels=[0, 1],
								average="macro")
	recall = recall_score(train_test_y, y_pred, labels=[0, 1], average="macro")
	print(
		f"f1 score: {f1}, precision score: {precision}, recall score: {recall}.")
	print(confusion_matrix(train_test_y, y_pred, labels=[0, 1]))
	print()
# scores.append(f1)

# for score in scores:
#     print(score)
# print("max score:", max(scores))

# for threshold in range(5,10):
#     y_result = (y_prob > threshold).astype(int)
#     scores.append((f1, precision, recall))
# print()
#
# plot_data = [[], [], [], [], []]
# for i, num_of_trees in enumerate(tree_range):
#     for j, max_depth in enumerate(depth_range):
#         plot_data[0].append(num_of_trees)
#         plot_data[1].append(max_depth)
#         plot_data[2].append(scores[i][j][0])
#         plot_data[3].append(scores[i][j][1])
#         plot_data[4].append(scores[i][j][2])
#
# print("plotting...")
# go.Figure(data=[go.Scatter3d(x=plot_data[0], z=plot_data[1], y=plot_data[2],
#                              mode='markers',
#                              marker=dict(color=plot_data[2], reversescale=True,
#                                          showscale=True, size=3))],
#           layout=go.Layout(
#               title=r"$\text{f1 scores}$",
#               scene_aspectmode="cube", showlegend=False,
#               scene=dict(xaxis_title="num_of_trees",
#                          yaxis_title="max_depth",
#                          zaxis_title="f1_score",
#                          camera=dict(eye=dict(x=1, y=-1.8,
#                                               z=.1))))).show()

f1 score: 0.5231143552311436, precision score: 0.6070402298850575, recall score: 0.5170052499429354.
[[671   3]
 [ 25   1]]
f1 score: 0.48867786705624544, precision score: 0.481294964028777, recall score: 0.49629080118694363.
[[669   5]
 [ 26   0]]

f1 score: 0.5231143552311436, precision score: 0.6070402298850575, recall score: 0.5170052499429354.
[[671   3]
 [ 25   1]]
f1 score: 0.48867786705624544, precision score: 0.481294964028777, recall score: 0.49629080118694363.
[[669   5]
 [ 26   0]]

f1 score: 0.5231143552311436, precision score: 0.6070402298850575, recall score: 0.5170052499429354.
[[671   3]
 [ 25   1]]
f1 score: 0.48867786705624544, precision score: 0.481294964028777, recall score: 0.49629080118694363.
[[669   5]
 [ 26   0]]

f1 score: 0.5231143552311436, precision score: 0.6070402298850575, recall score: 0.5170052499429354.
[[671   3]
 [ 25   1]]
f1 score: 0.48867786705624544, precision score: 0.481294964028777, recall score: 0.49629080118694363.
[[669   5]
 [ 26   0]]

